In [8]:
from src.luxai_s3.wrappers import LuxAIS3GymEnv
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import math
from collections import deque
from torch.distributions import Categorical
from tqdm import tqdm

In [72]:
torch.nan_to_num((torch.zeros(1)+1e-18) * (-torch.inf))

tensor([-3.4028e+38])

In [119]:
ac = torch.from_numpy(np.array([[0,12,56],[5,13,45],[4,13,4],[5,2,3]]))
a = ac[:,0]!=5

ac[a,2] = 0
ac[a,1] = 0
print(ac)

tensor([[ 0,  0,  0],
        [ 5, 13, 45],
        [ 4,  0,  0],
        [ 5,  2,  3]])


In [117]:
a = torch.zeros(5,2,6)
a[:,0] = 1
print(a)
a = torch.sum(a.flatten(0,1),axis=1)
print(a)
print(a.view(5,2))

tensor([[[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.]]])
tensor([6., 0., 6., 0., 6., 0., 6., 0., 6., 0.])
tensor([[6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.],
        [6., 0.]])


In [108]:
indices = torch.randperm(100).type(torch.IntTensor)
x = torch.rand((100))
x[indices[:100]]

tensor([0.9622, 0.5488, 0.1107, 0.0701, 0.7722, 0.5672, 0.3098, 0.7879, 0.5168,
        0.2152, 0.2722, 0.0164, 0.0052, 0.8149, 0.0077, 0.0638, 0.8164, 0.4512,
        0.0756, 0.6845, 0.3602, 0.6918, 0.2439, 0.1124, 0.7072, 0.1796, 0.3955,
        0.7336, 0.0934, 0.8547, 0.4365, 0.4686, 0.5946, 0.4098, 0.1832, 0.0687,
        0.1274, 0.3494, 0.0645, 0.9324, 0.2774, 0.6857, 0.1230, 0.1161, 0.7086,
        0.2544, 0.7184, 0.3671, 0.8880, 0.0355, 0.6036, 0.3076, 0.7279, 0.5499,
        0.0059, 0.0276, 0.7266, 0.2689, 0.1002, 0.4770, 0.9395, 0.6968, 0.2521,
        0.9680, 0.2555, 0.2303, 0.7571, 0.4251, 0.9588, 0.0950, 0.6509, 0.3164,
        0.7800, 0.7587, 0.1279, 0.5298, 0.8548, 0.6286, 0.9869, 0.9008, 0.0409,
        0.8541, 0.0707, 0.0888, 0.5810, 0.5028, 0.8040, 0.0835, 0.2401, 0.3443,
        0.7184, 0.0859, 0.8994, 0.6936, 0.7999, 0.7590, 0.3733, 0.1008, 0.2849,
        0.9459])

In [30]:
Categorical(probs=torch.Tensor([[0.3,0.3,0.3]])).sample()

tensor([0])

In [74]:
env = LuxAIS3GymEnv(numpy_output=True)
obs,_ = env.reset()
print(env.env_params)


EnvParams(max_steps_in_match=100, map_type=1, map_width=24, map_height=24, num_teams=2, match_count_per_episode=5, max_units=16, init_unit_energy=100, min_unit_energy=0, max_unit_energy=400, unit_move_cost=4, spawn_rate=3, unit_sap_cost=46, unit_sap_range=4, unit_sap_dropoff_factor=0.5, unit_energy_void_factor=0.125, max_energy_nodes=6, max_energy_per_tile=20, min_energy_per_tile=-20, max_relic_nodes=6, relic_config_size=5, fog_of_war=True, unit_sensor_range=2, nebula_tile_vision_reduction=0, nebula_tile_energy_reduction=0, nebula_tile_drift_speed=0.02500000037252903, energy_node_drift_speed=0.029999999329447746, energy_node_drift_magnitude=5)


In [298]:
env = LuxAIS3GymEnv(numpy_output=True)
obs,_ = env.reset()


for i in tqdm(range(20)) :
    action = dict(
                player_0=np.random.randint(0,4,size=(env.env_params.max_units, 3)),
                player_1=np.random.randint(0,4,size=(env.env_params.max_units, 3))
            )
    obs, reward, terminated, truncated, info = env.step(action)

print(obs['player_0']['units']['position'][0])

action_0 = np.zeros((16,3),dtype=int)
action_1 = np.zeros((16,3),dtype=int)

action_0[0,0] = 3
action = dict(
                player_0=action_0,
                player_1=action_1
            )
obs, reward, terminated, truncated, info = env.step(action)
print(obs['player_0']['units']['position'][0])


100%|██████████| 20/20 [00:04<00:00,  4.60it/s]

[[ 4  1]
 [ 4  1]
 [ 4  0]
 [ 3  0]
 [ 2  0]
 [ 2  0]
 [ 0  1]
 [-1 -1]
 [-1 -1]
 [-1 -1]
 [-1 -1]
 [-1 -1]
 [-1 -1]
 [-1 -1]
 [-1 -1]
 [-1 -1]]
[[ 4  2]
 [ 4  1]
 [ 4  0]
 [ 3  0]
 [ 2  0]
 [ 2  0]
 [ 0  1]
 [-1 -1]
 [-1 -1]
 [-1 -1]
 [-1 -1]
 [-1 -1]
 [-1 -1]
 [-1 -1]
 [-1 -1]
 [-1 -1]]


In [75]:
env = LuxAIS3GymEnv(numpy_output=True)
env.reset()
for i in tqdm(range(50)) :
    action = dict(
                player_0=np.random.randint(0,5,size=(env.env_params.max_units, 3)),
                player_1=np.random.randint(0,5,size=(env.env_params.max_units, 3))
            )
    obs, reward, terminated, truncated, info = env.step(action)
print(obs)
    

100%|██████████| 50/50 [00:01<00:00, 28.15it/s]

{'player_0': {'units': {'position': array([[[ 1,  6],
        [ 1,  0],
        [ 1,  5],
        [ 0,  2],
        [ 2,  0],
        [ 0,  2],
        [ 0,  2],
        [ 0,  1],
        [ 1,  1],
        [ 0,  1],
        [ 4,  2],
        [ 1,  3],
        [ 2,  3],
        [ 3,  0],
        [ 0,  1],
        [ 0,  0]],

       [[-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1],
        [-1, -1]]], dtype=int16), 'energy': array([[  9,   0,  70,  43,   0,   0,   0,   0,   0,   6, 101,  71, 103,
         62,  64,  92],
       [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1]], dtype=int16)}, 'units_mask': array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       

In [113]:
torch.where(torch.tensor([[True,False,True],[False,True,True]]))

(tensor([0, 0, 1, 1]), tensor([0, 2, 1, 2]))

In [276]:
class Policy(nn.Module) :

    def __init__(self,n_input,n_action,env_param,player) :

        super(Policy,self).__init__()

        if player == 'player_0' :
            self.player_id = 0
        else :
            self.player_id = 1

        self.n_action = n_action
        self.n_units = env_param.max_units
        self.sap_range = env_param.unit_sap_range
        self.unit_move_cost = env_param.unit_move_cost
        self.unit_sap_cost = env_param.unit_sap_cost

        self.feature_size = 64
        self.network_size = [1024,512]

        self.hidden = [nn.Linear(self.network_size[i],self.network_size[i+1],dtype=torch.double) for i in range(len(self.network_size)-1)]
        self.inputs = nn.Linear(n_input,self.network_size[0],dtype=torch.double)
        self.outputs = nn.Linear(self.network_size[-1],self.feature_size,dtype=torch.double)

        self.actor_action = nn.Linear(self.feature_size,self.n_action*self.n_units,dtype=torch.double)
        self.actor_dx = nn.Linear(self.feature_size,(self.sap_range*2+1)*self.n_units,dtype=torch.double)
        self.actor_dy = nn.Linear(self.feature_size,(self.sap_range*2+1)*self.n_units,dtype=torch.double)

        self.critic = nn.Linear(self.feature_size,1,dtype=torch.double)

    def training_forward(self,x,action,mask_action,mask_dx,mask_dy) :

        x = F.relu(self.inputs(x))
        for layer in self.hidden :
            x = F.relu(layer(x))
        x = F.relu(self.outputs(x))

        actor_action = self.actor_action(x).view(-1,self.n_units,self.n_action) + torch.nan_to_num(mask_action*(-torch.inf))
        actor_dx = self.actor_dx(x).view(-1,self.n_units,self.sap_range*2+1) + torch.nan_to_num(mask_dx*(-torch.inf))
        actor_dy = self.actor_dy(x).view(-1,self.n_units,self.sap_range*2+1) + torch.nan_to_num(mask_dy*(-torch.inf))

        actor_action = F.log_softmax(actor_action,dim=-1)
        actor_dx = F.log_softmax(actor_dx,dim=-1)
        actor_dy = F.log_softmax(actor_dy,dim=-1)

        value = self.critic(x)

        # Computing log probabilities for the actions

        batch_size = actor_action.size(0)
        n_units = actor_action.size(1)

        step_indices = torch.arange(batch_size).unsqueeze(1).expand(-1, n_units)
        unit_indices = torch.arange(n_units).unsqueeze(0).expand(batch_size, -1) 

        log_prob = torch.sum(actor_action[step_indices,unit_indices, action[:,:, 0]],axis=1)
        log_prob += torch.sum(actor_dx[step_indices,unit_indices, action[:,:, 1]+self.sap_range],axis=1)
        log_prob += torch.sum(actor_dy[step_indices,unit_indices, action[:,:, 2]+self.sap_range],axis=1)

        return value,log_prob.view(-1,1)

    def forward(self,x,obs) :

        x = F.relu(self.inputs(x))
        for layer in self.hidden :
            x = F.relu(layer(x))
        x = F.relu(self.outputs(x))

        state = {}
        state['energy'] = torch.from_numpy(obs['units']['energy'][self.player_id].astype(int)).view(self.n_units)
        state['units'] = torch.from_numpy(obs['units']['position'][self.player_id].astype(int)).view(self.n_units,2)
        state['map'] = torch.from_numpy(obs['map_features']['tile_type'].astype(int)).view(24,24)
        

        energy_mask = state['energy'] < self.unit_move_cost
        sap_mask = state['energy'] < self.unit_sap_cost

        mask_action = torch.zeros(self.n_units,self.n_action)
        mask_dx = torch.zeros(self.n_units,self.sap_range*2+1)
        mask_dy = torch.zeros(self.n_units,self.sap_range*2+1)

        mask_action[torch.where(energy_mask)[0],1:] += 1
        mask_action[torch.where(sap_mask)[0],-1] += 1

        directions = torch.tensor([[0,-1],[1,0],[0,1],[-1,0],]).view(4,2)
        target_tiles = state['units'].unsqueeze(1).expand(self.n_units, 4, 2) + directions
        clamp_target_tiles = torch.clamp(target_tiles,0,23).view(self.n_units*4,2)
        target_tiles_type = state['map'][clamp_target_tiles[:,0],clamp_target_tiles[:,1]].view(self.n_units,4)

        correct_move_direction = (((target_tiles >= 0) & (target_tiles <= 23)).all(dim=-1)) & (target_tiles_type != 1)
        forbidden_move = 1 - correct_move_direction.int() 
        mask_action[:,1:-1] += forbidden_move
        
        
        actor_action = self.actor_action(x).view(self.n_units,self.n_action) + torch.nan_to_num(mask_action*(-torch.inf))
        actor_action = F.log_softmax(actor_action,dim=-1)
        action_choice = Categorical(logits=actor_action).sample()

        

        sap_mask =  sap_mask | (action_choice !=5)

        mask_dx[torch.where(sap_mask)[0]] = 1
        mask_dy[torch.where(sap_mask)[0]] = 1
        mask_dx[:,self.sap_range] = 0
        mask_dy[:,self.sap_range] = 0

        actor_dx = self.actor_dx(x).view(self.n_units,self.sap_range*2+1) + torch.nan_to_num(mask_dx*(-torch.inf))
        actor_dy = self.actor_dy(x).view(self.n_units,self.sap_range*2+1) + torch.nan_to_num(mask_dy*(-torch.inf))
        actor_dx = F.log_softmax(actor_dx,dim=-1)
        actor_dy = F.log_softmax(actor_dy,dim=-1)

        # Sampling action based on the policy
        action = torch.zeros(self.n_units, 3, dtype=torch.int)

        action[:, 0] = action_choice
        action[:, 1] = Categorical(logits=actor_dx).sample() - self.sap_range
        action[:, 2] = Categorical(logits=actor_dy).sample() - self.sap_range

        value = self.critic(x)

        return action, value, mask_action, mask_dx, mask_dy 
    
model = Policy(1880,6,env.env_params,'player_0')
state = torch.randn(1880,dtype=torch.double)

action , value, mask_action , mask_dx, mask_dy  = model(state,obs['player_0'])

model.training_forward( state.unsqueeze(0).expand(10,-1),
                        action.unsqueeze(0).expand(10,-1,-1),
                        mask_action.unsqueeze(0).expand(10,-1,-1) ,
                        mask_dx.unsqueeze(0).expand(10,-1,-1),
                        mask_dy.unsqueeze(0).expand(10,-1,-1))


(tensor([[0.0306],
         [0.0306],
         [0.0306],
         [0.0306],
         [0.0306],
         [0.0306],
         [0.0306],
         [0.0306],
         [0.0306],
         [0.0306]], dtype=torch.float64, grad_fn=<AddmmBackward0>),
 tensor([[-16.0278],
         [-16.0278],
         [-16.0278],
         [-16.0278],
         [-16.0278],
         [-16.0278],
         [-16.0278],
         [-16.0278],
         [-16.0278],
         [-16.0278]], dtype=torch.float64, grad_fn=<ViewBackward0>))

In [33]:
def obs_to_state(obs) :
    list_state = []

    #Units
    list_state.append(torch.from_numpy(obs['units']['position'].astype(float)).flatten()/24) #position
    list_state.append(torch.from_numpy(obs['units']['energy'].astype(float)).flatten()/400) #energy
    list_state.append(torch.from_numpy(obs['units_mask'].astype(float)).flatten()) #unit_mask

    #Game
    list_state.append(torch.from_numpy(obs['team_points'].astype(float)).flatten()/3000) #team_points
    list_state.append(torch.from_numpy(obs['team_wins'].astype(float)).flatten()/3) #team_wins

    #Map
    print(obs['sensor_mask'].astype(float))
    print(obs['map_features']['energy'].astype(float))
    print(obs['map_features']['tile_type'].astype(float))
    print(obs['relic_nodes'].astype(float))
    print(obs['relic_nodes_mask'].astype(float))
    list_state.append(torch.from_numpy(obs['sensor_mask'].astype(float)).flatten()) #sensor_mask
    list_state.append(torch.from_numpy(obs['map_features']['energy'].astype(float)).flatten()) #map_energy
    list_state.append(torch.from_numpy(obs['map_features']['tile_type'].astype(float)).flatten()) #map_tile_type

    list_state.append(torch.from_numpy(obs['relic_nodes'].astype(float)).flatten()) #relic_nodes
    list_state.append(torch.from_numpy(obs['relic_nodes_mask'].astype(float)).flatten()) #relic_nodes_mask
    
    list_state.append(torch.from_numpy(obs['steps'].astype(float)).flatten()/100) #steps
    list_state.append(torch.from_numpy(obs['match_steps'].astype(float)).flatten()/5) #match_steps

    return torch.cat(list_state)

state0 = obs_to_state(obs['player_0'])
state1 = obs_to_state(obs['player_1'])

print(torch.mean(state0),torch.std(state0))

[[1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.